<a href="https://colab.research.google.com/github/zakariazemmahi/waste-detection-yolov8/blob/main/Models/Application_de_comptur_vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **⚙️ Étape 1 – Installer les dépendances**

In [2]:
!pip install streamlit ultralytics
!npm install -g localtunnel


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/2

# **📝 Étape 2 – Créer ton fichier app.py Streamlit**

In [7]:
%%writefile app.py
import streamlit as st
from ultralytics import YOLO
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import requests
import streamlit.components.v1 as components
import cv2
from io import BytesIO
import base64
import plotly.express as px
import plotly.graph_objects as go
import gc
import time

# Configuration de la page
st.set_page_config(
    page_title="SmartWasteDetection",
    page_icon="♻️",
    layout="wide",
    initial_sidebar_state="expanded"
)

# CSS personnalisé moderne
st.markdown("""
<style>
    @import url('https://fonts.googleapis.com/css2?family=Poppins:wght@300;400;600;700&display=swap');

    * {
        font-family: 'Poppins', sans-serif;
    }

    .main-header {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        padding: 3rem 2rem;
        border-radius: 20px;
        text-align: center;
        margin-bottom: 2rem;
        box-shadow: 0 20px 40px rgba(0, 0, 0, 0.1);
        position: relative;
        overflow: hidden;
    }

    .main-header::before {
        content: '';
        position: absolute;
        top: -50%;
        left: -50%;
        width: 200%;
        height: 200%;
        background: linear-gradient(45deg, transparent, rgba(255,255,255,0.1), transparent);
        animation: shine 3s infinite;
    }

    @keyframes shine {
        0% { transform: translateX(-100%) translateY(-100%) rotate(45deg); }
        100% { transform: translateX(100%) translateY(100%) rotate(45deg); }
    }

    .main-header h1 {
        color: white;
        font-size: 3.5rem;
        margin: 0;
        font-weight: 700;
        text-shadow: 2px 2px 4px rgba(0,0,0,0.3);
        position: relative;
        z-index: 1;
    }

    .main-header p {
        color: rgba(255,255,255,0.9);
        font-size: 1.3rem;
        margin-top: 1rem;
        position: relative;
        z-index: 1;
        font-weight: 300;
    }

    .detection-card {
        background: linear-gradient(145deg, #ffffff, #f0f2f6);
        padding: 2rem;
        border-radius: 20px;
        box-shadow: 0 10px 30px rgba(0, 0, 0, 0.1);
        border: 1px solid rgba(255,255,255,0.2);
        margin: 1rem 0;
        transition: transform 0.3s ease, box-shadow 0.3s ease;
    }

    .detection-card:hover {
        transform: translateY(-5px);
        box-shadow: 0 20px 40px rgba(0, 0, 0, 0.15);
    }

    .stat-card {
        background: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%);
        padding: 2rem;
        border-radius: 15px;
        text-align: center;
        color: white;
        box-shadow: 0 10px 25px rgba(79, 172, 254, 0.3);
        transition: transform 0.3s ease;
    }

    .stat-card:hover {
        transform: scale(1.05);
    }

    .stat-number {
        font-size: 2.5rem;
        font-weight: 700;
        margin: 0;
    }

    .stat-label {
        font-size: 0.9rem;
        opacity: 0.9;
        font-weight: 300;
    }

    .upload-zone {
        border: 3px dashed #667eea;
        border-radius: 20px;
        padding: 3rem;
        text-align: center;
        background: linear-gradient(135deg, rgba(102, 126, 234, 0.1), rgba(118, 75, 162, 0.1));
        margin: 2rem 0;
        transition: all 0.3s ease;
    }

    .upload-zone:hover {
        border-color: #764ba2;
        background: linear-gradient(135deg, rgba(102, 126, 234, 0.2), rgba(118, 75, 162, 0.2));
    }

    .sidebar-info {
        background: linear-gradient(135deg, #ff9a56 0%, #ff6b95 100%);
        padding: 1.5rem;
        border-radius: 15px;
        color: white;
        text-align: center;
        margin: 1rem 0;
    }

    .progress-container {
        background: rgba(255,255,255,0.1);
        border-radius: 10px;
        padding: 1rem;
        margin: 1rem 0;
    }

    .image-container {
        border-radius: 15px;
        overflow: hidden;
        box-shadow: 0 10px 30px rgba(0, 0, 0, 0.2);
        margin: 1rem 0;
        transition: transform 0.3s ease;
    }

    .image-container:hover {
        transform: scale(1.02);
    }

    .results-section {
        background: linear-gradient(135deg, #ffecd2 0%, #fcb69f 100%);
        padding: 2rem;
        border-radius: 20px;
        margin: 2rem 0;
    }

    .waste-type-badge {
        display: inline-block;
        padding: 0.5rem 1rem;
        background: linear-gradient(135deg, #ff6b6b, #ee5a24);
        color: white;
        border-radius: 20px;
        margin: 0.25rem;
        font-weight: 600;
        box-shadow: 0 5px 15px rgba(255, 107, 107, 0.3);
    }

    .non-waste-badge {
        display: inline-block;
        padding: 0.5rem 1rem;
        background: linear-gradient(135deg, #00d2d3, #54a0ff);
        color: white;
        border-radius: 20px;
        margin: 0.25rem;
        font-weight: 600;
        box-shadow: 0 5px 15px rgba(84, 160, 255, 0.3);
    }

    .stButton > button {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        color: white;
        border: none;
        padding: 0.75rem 2rem;
        border-radius: 25px;
        font-weight: 600;
        transition: all 0.3s ease;
        box-shadow: 0 5px 15px rgba(102, 126, 234, 0.3);
    }

    .stButton > button:hover {
        transform: translateY(-2px);
        box-shadow: 0 10px 25px rgba(102, 126, 234, 0.4);
    }

    .analysis-header {
        text-align: center;
        color: #333;
        font-size: 2rem;
        font-weight: 600;
        margin: 2rem 0;
        background: linear-gradient(135deg, #667eea, #764ba2);
        -webkit-background-clip: text;
        -webkit-text-fill-color: transparent;
        background-clip: text;
    }
</style>
""", unsafe_allow_html=True)

@st.cache_resource
def load_models():
    """Charge les modèles YOLO avec gestion d'erreur améliorée"""
    try:
        with st.spinner("🤖 Chargement du modèle de détection..."):
            model_detect = YOLO("/content/drive/MyDrive/yolov8_best_smartdetection.pt")

        with st.spinner("🔍 Chargement du modèle de classification..."):
            model_classify = YOLO("/content/drive/MyDrive/yolov8_best.pt")

        return model_detect, model_classify
    except Exception as e:
        st.error(f"❌ Erreur lors du chargement des modèles : {str(e)}")
        return None, None

def process_image_detection(image, model_detect):
    """Processus de détection des objets (déchets vs non-déchets)"""
    img_array = np.array(image)
    results = model_detect.predict(img_array, conf=0.25, verbose=False)

    waste_objects = []
    non_waste_objects = []

    if results and results[0].boxes is not None:
        for box in results[0].boxes:
            cls_id = int(box.cls)
            class_name = results[0].names[cls_id]
            conf = float(box.conf)
            bbox = box.xyxy.cpu().numpy().astype(int)[0]

            obj = {
                'class': class_name,
                'confidence': conf,
                'bbox': bbox
            }

            # Vérification si l'objet est un déchet
            if class_name.lower() == "dechet":
                waste_objects.append(obj)
            else:
                non_waste_objects.append(obj)

    return waste_objects, non_waste_objects, results[0] if results else None

def classify_waste(image, bbox, model_classify):
    """Classification du type de déchet pour les objets détectés comme déchets"""
    x1, y1, x2, y2 = bbox

    # Extraction de la région d'intérêt
    cropped = np.array(image)[y1:y2, x1:x2]

    if cropped.size == 0:
        return None, 0

    # Prédiction sur la région extraite
    results = model_classify.predict(cropped, conf=0.25, verbose=False)

    if results and results[0].boxes is not None and len(results[0].boxes) > 0:
        box = results[0].boxes[0]
        class_name = results[0].names[int(box.cls)]
        confidence = float(box.conf)
        return class_name, confidence

    return None, 0

def create_annotated_image(image, waste_objects, non_waste_objects, classifications):
    """Création d'une image annotée avec les détections"""
    # Créer une copie pour éviter de modifier l'original
    annotated_image = image.copy()
    draw = ImageDraw.Draw(annotated_image)

    # Charger une police
    try:
        font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", 24)
        font_small = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 18)
    except:
        font = ImageFont.load_default()
        font_small = ImageFont.load_default()

    # Annoter les déchets (rouge) avec leur classification
    for i, obj in enumerate(waste_objects):
        x1, y1, x2, y2 = obj['bbox']

        # Rectangle rouge pour les déchets
        draw.rectangle([x1, y1, x2, y2], outline="red", width=4)

        # Texte avec classification si disponible
        if i < len(classifications) and classifications[i][0]:
            waste_type = classifications[i][0]
            confidence = classifications[i][1]
            label = f"{waste_type} ({confidence:.0%})"
        else:
            label = f"Déchet ({obj['confidence']:.0%})"

        # Fond pour le texte
        text_bbox = draw.textbbox((x1, y1-30), label, font=font)
        draw.rectangle(text_bbox, fill="red")
        draw.text((x1, y1-30), label, fill="white", font=font)

    # Annoter les non-déchets (vert)
    for obj in non_waste_objects:
        x1, y1, x2, y2 = obj['bbox']

        # Rectangle vert pour les non-déchets
        draw.rectangle([x1, y1, x2, y2], outline="green", width=4)

        label = f"{obj['class']} ({obj['confidence']:.0%})"

        # Fond pour le texte
        text_bbox = draw.textbbox((x1, y1-30), label, font=font)
        draw.rectangle(text_bbox, fill="green")
        draw.text((x1, y1-30), label, fill="white", font=font)

    return annotated_image

def display_detection_results(waste_objects, non_waste_objects, classifications):
    """Affichage détaillé des résultats de détection"""

    st.markdown("### 📊 Résultats de l'analyse")

    col1, col2 = st.columns(2)

    with col1:
        st.markdown("#### 🗑️ Déchets détectés")
        if waste_objects:
            for i, obj in enumerate(waste_objects):
                with st.expander(f"Déchet #{i+1} - Confiance: {obj['confidence']:.0%}"):
                    if i < len(classifications) and classifications[i][0]:
                        st.markdown(f"**Type:** {classifications[i][0]}")
                        st.markdown(f"**Confiance classification:** {classifications[i][1]:.0%}")
                    else:
                        st.markdown("**Type:** Non classifié")

                    x1, y1, x2, y2 = obj['bbox']
                    st.markdown(f"**Position:** ({x1}, {y1}) → ({x2}, {y2})")
        else:
            st.info("Aucun déchet détecté dans cette image.")

    with col2:
        st.markdown("#### ✅ Objets non-déchets")
        if non_waste_objects:
            for i, obj in enumerate(non_waste_objects):
                with st.expander(f"Objet #{i+1} - {obj['class']}"):
                    st.markdown(f"**Confiance:** {obj['confidence']:.0%}")
                    x1, y1, x2, y2 = obj['bbox']
                    st.markdown(f"**Position:** ({x1}, {y1}) → ({x2}, {y2})")
        else:
            st.info("Aucun objet non-déchet détecté.")

# Interface principale
def main():
    # Sidebar améliorée
    with st.sidebar:
        st.markdown("""
        <div class="sidebar-info">
            <h2>🤖 Smart AI Detection</h2>
            <p>ENSAM Meknès - 2025</p>
        </div>
        """, unsafe_allow_html=True)

        st.markdown("---")

        st.markdown("### 🔧 Paramètres")
        confidence_threshold = st.slider("Seuil de confiance", 0.1, 1.0, 0.25, 0.05)
        show_details = st.checkbox("Afficher les détails", value=True)

        st.markdown("---")

        st.markdown("### 📋 Instructions")
        st.markdown("""
        1. 📤 Uploadez vos images
        2. 🔍 Le système détecte les objets
        3. 🗑️ Classification automatique des déchets
        4. 📊 Visualisation des résultats
        """)

        st.markdown("---")

        st.markdown("### 🎯 Fonctionnalités")
        st.markdown("""
        - **Détection IA** : YOLOv8 avancé
        - **Classification** : Types de déchets
        - **Visualisation** : Graphiques interactifs
        - **Export** : Images annotées
        """)

    # Header principal
    st.markdown("""
    <div class="main-header">
        <h1>🚀 Smart Waste Detection</h1>
        <p>Intelligence Artificielle pour la détection et classification des déchets</p>
    </div>
    """, unsafe_allow_html=True)

    # Chargement des modèles
    model_detect, model_classify = load_models()

    if model_detect is None or model_classify is None:
        st.error("Impossible de charger les modèles. Vérifiez les chemins des fichiers.")
        st.stop()

    st.success("✅ Modèles chargés avec succès !")

    # Zone d'upload
    st.markdown('<div class="upload-zone">', unsafe_allow_html=True)
    uploaded_files = st.file_uploader(
        "📤 Glissez-déposez vos images ici ou cliquez pour parcourir",
        type=["jpg", "jpeg", "png"],
        accept_multiple_files=True,
        help="Formats supportés: JPG, JPEG, PNG"
    )
    st.markdown('</div>', unsafe_allow_html=True)

    if uploaded_files:
        # Statistiques globales
        total_images = len(uploaded_files)
        total_detected = 0
        total_waste = 0
        total_non_waste = 0
        all_waste_types = []

        # Barre de progression
        progress_bar = st.progress(0)
        status_text = st.empty()

        # Conteneur pour les résultats
        results_container = st.container()

        # Traitement de chaque image
        for idx, uploaded_file in enumerate(uploaded_files):
            # Mise à jour de la progression
            progress = (idx + 1) / total_images
            progress_bar.progress(progress)
            status_text.text(f"Traitement de l'image {idx + 1}/{total_images}: {uploaded_file.name}")

            # Chargement de l'image
            image = Image.open(uploaded_file).convert("RGB")

            with results_container:
                st.markdown(f'<div class="analysis-header">📸 Analyse: {uploaded_file.name}</div>', unsafe_allow_html=True)

                # Affichage image originale
                col1, col2 = st.columns(2)

                with col1:
                    st.markdown("#### Image originale")
                    st.markdown('<div class="image-container">', unsafe_allow_html=True)
                    st.image(image, use_column_width=True)
                    st.markdown('</div>', unsafe_allow_html=True)

                # Traitement IA
                with st.spinner("🤖 Analyse IA en cours..."):
                    # 1. Détection des objets (déchets vs non-déchets)
                    waste_objects, non_waste_objects, detection_result = process_image_detection(image, model_detect)

                    # 2. Classification des déchets détectés
                    classifications = []
                    for waste_obj in waste_objects:
                        waste_type, confidence = classify_waste(image, waste_obj['bbox'], model_classify)
                        classifications.append((waste_type, confidence))
                        if waste_type:
                            all_waste_types.append(waste_type)

                    # 3. Création de l'image annotée
                    annotated_image = create_annotated_image(image, waste_objects, non_waste_objects, classifications)

                with col2:
                    st.markdown("#### Image analysée")
                    st.markdown('<div class="image-container">', unsafe_allow_html=True)
                    st.image(annotated_image, use_column_width=True)
                    st.markdown('</div>', unsafe_allow_html=True)

                # Statistiques pour cette image
                st.markdown('<div class="detection-card">', unsafe_allow_html=True)
                cols = st.columns(4)

                with cols[0]:
                    st.markdown(f"""
                    <div class="stat-card">
                        <div class="stat-number">{len(waste_objects) + len(non_waste_objects)}</div>
                        <div class="stat-label">Objets détectés</div>
                    </div>
                    """, unsafe_allow_html=True)

                with cols[1]:
                    st.markdown(f"""
                    <div class="stat-card" style="background: linear-gradient(135deg, #ff6b6b 0%, #ee5a24 100%);">
                        <div class="stat-number">{len(waste_objects)}</div>
                        <div class="stat-label">Déchets</div>
                    </div>
                    """, unsafe_allow_html=True)

                with cols[2]:
                    st.markdown(f"""
                    <div class="stat-card" style="background: linear-gradient(135deg, #00d2d3 0%, #54a0ff 100%);">
                        <div class="stat-number">{len(non_waste_objects)}</div>
                        <div class="stat-label">Non-déchets</div>
                    </div>
                    """, unsafe_allow_html=True)

                with cols[3]:
                    classified_waste = sum(1 for c in classifications if c[0] is not None)
                    st.markdown(f"""
                    <div class="stat-card" style="background: linear-gradient(135deg, #ffa726 0%, #fb8c00 100%);">
                        <div class="stat-number">{classified_waste}</div>
                        <div class="stat-label">Classifiés</div>
                    </div>
                    """, unsafe_allow_html=True)

                st.markdown('</div>', unsafe_allow_html=True)

                # Détails des détections
                if show_details:
                    display_detection_results(waste_objects, non_waste_objects, classifications)

                # Bouton de téléchargement
                buf = BytesIO()
                annotated_image.save(buf, format="PNG")
                st.download_button(
                    label="📥 Télécharger l'image annotée",
                    data=buf.getvalue(),
                    file_name=f"analysed_{uploaded_file.name}",
                    mime="image/png"
                )

                # Mise à jour des totaux
                total_detected += len(waste_objects) + len(non_waste_objects)
                total_waste += len(waste_objects)
                total_non_waste += len(non_waste_objects)

                st.markdown("---")

        # Finalisation de la barre de progression
        progress_bar.progress(1.0)
        status_text.text("✅ Traitement terminé !")

        # Résumé global
        st.markdown('<div class="results-section">', unsafe_allow_html=True)
        st.markdown("## 📈 Statistiques globales")

        col1, col2, col3, col4 = st.columns(4)

        with col1:
            st.metric("📸 Images traitées", total_images)
        with col2:
            st.metric("🔍 Objets détectés", total_detected)
        with col3:
            st.metric("🗑️ Déchets trouvés", total_waste)
        with col4:
            st.metric("✅ Non-déchets", total_non_waste)

        # Graphiques de synthèse
        if all_waste_types:
            st.markdown("### 📊 Distribution des types de déchets")

            # Compter les occurrences
            waste_counts = {}
            for waste_type in all_waste_types:
                waste_counts[waste_type] = waste_counts.get(waste_type, 0) + 1

            # Graphique en barres
            fig_bar = px.bar(
                x=list(waste_counts.keys()),
                y=list(waste_counts.values()),
                title="Types de déchets détectés",
                labels={'x': 'Type de déchet', 'y': 'Nombre d\'occurrences'},
                color=list(waste_counts.values()),
                color_continuous_scale='Viridis'
            )
            fig_bar.update_layout(showlegend=False)
            st.plotly_chart(fig_bar, use_container_width=True)

            # Graphique circulaire
            fig_pie = px.pie(
                values=list(waste_counts.values()),
                names=list(waste_counts.keys()),
                title="Répartition des types de déchets"
            )
            st.plotly_chart(fig_pie, use_container_width=True)

        st.markdown('</div>', unsafe_allow_html=True)

    else:
        # Message d'accueil
        st.markdown("""
        <div class="detection-card" style="text-align: center; padding: 3rem;">
            <h2>🎯 Prêt à analyser vos images ?</h2>
            <p style="font-size: 1.2rem; color: #666;">
                Uploadez vos images pour commencer l'analyse intelligente des déchets.
                Notre IA détectera automatiquement les objets et classifiera les déchets.
            </p>
            <br>
            <div style="display: flex; justify-content: center; gap: 2rem; flex-wrap: wrap;">
                <div style="text-align: center;">
                    <div style="font-size: 3rem;">🔍</div>
                    <strong>Détection</strong><br>
                    <small>Identifie les objets</small>
                </div>
                <div style="text-align: center;">
                    <div style="font-size: 3rem;">🗑️</div>
                    <strong>Classification</strong><br>
                    <small>Types de déchets</small>
                </div>
                <div style="text-align: center;">
                    <div style="font-size: 3rem;">📊</div>
                    <strong>Analyse</strong><br>
                    <small>Statistiques détaillées</small>
                </div>
            </div>
        </div>
        """, unsafe_allow_html=True)

if __name__ == "__main__":
    main()

Overwriting app.py


# **🌐 Étape 3 – Obtenir ton IP publique (optionnel)**

In [8]:
!wget -q -O - ipv4.icanhazip.com


35.194.254.95


# **🚀 Étape 4 – Lancer l'application et ouvrir un tunnel**

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501




⠙⠹⠸⠼
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.194.254.95:8501

⠴⠦your url is: https://funny-gifts-sip.loca.lt
2025-06-13 11:49:43.499 Examining the path of torch.classes raised:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/streamlit/web/bootstrap.py", line 347, in run
    if asyncio.get_running_loop().is_running():
       ^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: no running event loop

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/streamlit/watcher/local_sources_watcher.py", line 217, in get_module_paths
    potential_paths = extract_paths(module)
                      ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/streamlit/watcher/local_sources_watcher.py", line 210, in <lambda>
    lambda m: list(m.__path